In [1]:
import pandas as pd
import sqlite3

In [2]:
# Conexão database SQLite

consqlite = sqlite3.connect('C:/Users/matheus.candido/sqlite3/exploracao_petroleo_brasil.db', timeout=10)

In [3]:
# ETL Query f_exploracao

query_f_exploracao = f"""
SELECT POCO AS COD_POCO,
	   CADASTRO AS COD_CADASTRO_POCO,
	   ESTADO AS COD_ESTADO,
	   UPPER(REPLACE(SUBSTR(BACIA, 0, 3), 'ã', 'a') || LENGTH(BACIA) || REPLACE(REPLACE(SUBSTR(BACIA, -1, 1), 'á', 'a'), 'ó', 'o')) AS COD_BACIA,
	   UPPER(BLOCO) AS COD_BLOCO,
	   SUBSTR(CAMPO, -2, 3) || SUBSTR(CAMPO, 1, 3) || LENGTH(CAMPO) || SIG_CAMPO AS COD_CAMPO,
	   SIG_SONDA AS COD_SONDA,
	   TERRA_MAR,
	   CASE WHEN POCO_POS_ANP = 'S' THEN 1 ELSE 0 END POCO_POS_ANP,
	   TIPO AS COD_CATEGORIA,
	   SUBSTR(RECLASSIFICACAO, 0, 3) || LENGTH (RECLASSIFICACAO) || SUBSTR(RECLASSIFICACAO, -1, 1) AS COD_RECLASSIFICACAO,
	   CASE WHEN SITUACAO LIKE 'ABANDONADO TEMPORARIAMENTE SEM MONITORAMENTO' THEN 'AB44T' WHEN SITUACAO LIKE 'EM OBSERVAÇÃO' THEN 'EM13A' 
	   		WHEN SITUACAO LIKE 'EM COMPLETAÇÃO' THEN 'EM14A' WHEN SITUACAO LIKE 'EQUIPADO AGUARDANDO INÍCIO DE OPERAÇÃO' THEN 'EQ38A' 
	   		ELSE SUBSTR(SITUACAO, 0, 3) || LENGTH (SITUACAO) || SUBSTR(SITUACAO, -1, 1) END AS COD_SITUACAO,
	   SUBSTR(INICIO, 0, 11) AS DT_INICIO,
	   SUBSTR(TERMINO, 0, 11) AS DT_TERMINO,
	   SUBSTR(CONCLUSAO, 0, 11) AS DT_CONCLUSAO,
	   CASE WHEN TITULARIDADE LIKE 'Público' THEN 'P' WHEN TITULARIDADE LIKE 'Confidencial' THEN 'C' ELSE '' END AS COD_TITULARIDADE,
	   LATITUDE_BASE_DD,
	   LONGITUDE_BASE_DD, 
	   PROFUNDIDADE_VERTICAL_M,
	   PROFUNDIDADE_SONDADOR_M,
	   PROFUNDIDADE_MEDIDA_M,
	   REFERENCIA_DE_PROFUNDIDADE,
	   MESA_ROTATIVA,
	   COTA_ALTIMETRICA_M,
	   LAMINA_D_AGUA_M,
	   CASE WHEN CDPE LIKE 'Existe' THEN '1' ELSE '0' END CDPE,
	   CASE WHEN AGP LIKE 'Existe' THEN '1' ELSE '0' END AGP,
	   CASE WHEN PC LIKE 'Existe' THEN '1' ELSE '0' END PC,
	   CASE WHEN PAG LIKE 'Existe' THEN '1' ELSE '0' END PAG,
	   CASE WHEN PERFIS_CONVENCIONAIS LIKE 'Existe' THEN '1' ELSE '0' END PERFIS_CONVENCIONAIS,
	   CASE WHEN DURANTE_PERFURACAO LIKE 'Existe' THEN '1' ELSE '0' END DURANTE_PERFURACAO,
	   CASE WHEN PERFIS_DIGITAIS LIKE 'Existe' THEN '1' ELSE '0' END PERFIS_DIGITAIS,
	   CASE WHEN PERFIS_PROCESSADOS LIKE 'Existe' THEN '1' ELSE '0' END PERFIS_PROCESSADOS,
	   CASE WHEN PERFIS_ESPECIAIS LIKE 'Existe' THEN '1' ELSE '0' END PERFIS_ESPECIAIS,
	   CASE WHEN AMOSTRA_LATERAL LIKE 'Existe' THEN '1' ELSE '0' END AMOSTRA_LATERAL,
	   CASE WHEN SISMICA LIKE 'Existe' THEN '1' ELSE '0' END SISMICA,
	   CASE WHEN TABELA_TEMPO_PROFUNDIDADE LIKE 'Existe' THEN '1' ELSE '0' END TABELA_TEMPO_PROFUNDIDADE,
	   CASE WHEN DADOS_DIRECIONAIS LIKE 'Existe' THEN '1' ELSE '0' END DADOS_DIRECIONAIS,
	   CASE WHEN TESTE_A_CABO LIKE 'Existe' THEN '1' ELSE '0' END TESTE_A_CABO,
	   CASE WHEN TESTE_DE_FORMACAO LIKE 'Existe' THEN '1' ELSE '0' END TESTE_DE_FORMACAO,
	   CASE WHEN CANHONEIO LIKE 'Existe' THEN '1' ELSE '0' END CANHONEIO,
	   CASE WHEN TESTEMUNHO LIKE 'Existe' THEN '1' ELSE '0' END TESTEMUNHO,
	   CASE WHEN GEOQUIMICA LIKE 'Existe' THEN '1' ELSE '0' END GEOQUIMICA,
	   CASE WHEN ATINGIU_PRESAL LIKE 'Existe' THEN '1' ELSE '0' END ATINGIU_PRESAL,
	   SUBSTR(DHA_ATUALIZACAO, 0, 11) DT_ATUALIZACAO
FROM T_DATAHUB
"""

f_exploracao_df = pd.read_sql_query(query_f_exploracao, consqlite) 

In [4]:
# Informações de dataframe f_exploracao
f_exploracao_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30172 entries, 0 to 30171
Data columns (total 45 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   COD_POCO                    30172 non-null  object 
 1   COD_CADASTRO_POCO           30172 non-null  int64  
 2   COD_ESTADO                  30172 non-null  object 
 3   COD_BACIA                   30172 non-null  object 
 4   COD_BLOCO                   26759 non-null  object 
 5   COD_CAMPO                   26151 non-null  object 
 6   COD_SONDA                   30089 non-null  object 
 7   TERRA_MAR                   30172 non-null  object 
 8   POCO_POS_ANP                30172 non-null  int64  
 9   COD_CATEGORIA               30172 non-null  int64  
 10  COD_RECLASSIFICACAO         29012 non-null  object 
 11  COD_SITUACAO                30069 non-null  object 
 12  DT_INICIO                   30172 non-null  object 
 13  DT_TERMINO                  283

In [5]:
# Create table (DDL) SQLite f_exploracao
consqlite.execute('DROP TABLE F_EXPLORACAO')
consqlite.execute(f"""
CREATE TABLE IF NOT EXISTS F_EXPLORACAO (
	COD_POCO VARCHAR(20) PRIMARY KEY NOT NULL,
	COD_CADASTRO_POCO VARCHAR(50) NOT NULL,
	COD_ESTADO VARCHAR(10),
	COD_BACIA VARCHAR(10),
	COD_BLOCO VARCHAR(20),
	COD_CAMPO VARCHAR(20),
	COD_SONDA VARCHAR(10),
	TERRA_MAR VARCHAR(10),
	POCO_POS_ANP INT,
	COD_CATEGORIA INT,
	COD_RECLASSIFICACAO VARCHAR(20),
	COD_SITUACAO VARCHAR(20),
	DT_INICIO VARCHAR(50),
	DT_TERMINO VARCHAR(50),
	DT_CONCLUSAO VARCHAR(50),
	COD_TITULARIDADE VARCHAR(10),
	LATITUDE_BASE_DD REAL,
	LONGITUDE_BASE_DD REAL,
	PROFUNDIDADE_VERTICAL_M REAL,
	PROFUNDIDADE_SONDADOR_M REAL,
	PROFUNDIDADE_MEDIDA_M REAL,
	REFERENCIA_DE_PROFUNDIDADE VARCHAR(10),
	MESA_ROTATIVA REAL,
	COTA_ALTIMETRICA_M REAL,
	LAMINA_D_AGUA_M REAL,
	CDPE INT,
	AGP INT,
	PC INT,
	PAG INT,
	PERFIS_CONVENCIONAIS INT,
	DURANTE_PERFURACAO INT,
	PERFIS_DIGITAIS INT,
	PERFIS_PROCESSADOS INT,
	PERFIS_ESPECIAIS INT,
	AMOSTRA_LATERAL INT,
	SISMICA INT,
	TABELA_TEMPO_PROFUNDIDADE INT,
	DADOS_DIRECIONAIS INT,
	TESTE_A_CABO INT,
	TESTE_DE_FORMACAO INT,
	CANHONEIO INT,
	TESTEMUNHO INT,
	GEOQUIMICA INT,
	ATINGIU_PRESAL INT,
	DT_ATUALIZACAO VARCHAR(50),
	CONSTRAINT FK_F_EXPLORACAO_POCO
		FOREIGN KEY (COD_POCO) REFERENCES D_POCO_OPERADOR (COD_POCO)
	CONSTRAINT FK_F_EXPLORACAO_BACIA
		FOREIGN KEY (COD_BACIA) REFERENCES D_BACIA (COD_BACIA)
	CONSTRAINT FK_F_EXPLORACAO_CAMPO
		FOREIGN KEY (COD_CAMPO) REFERENCES D_CAMPO (COD_CAMPO)
	CONSTRAINT FK_F_EXPLORACAO_CATEGORIA
		FOREIGN KEY (COD_CATEGORIA) REFERENCES D_CATEGORIA (COD_CATEGORIA)
	CONSTRAINT FK_F_EXPLORACAO_ESTADO
		FOREIGN KEY (COD_ESTADO) REFERENCES D_ESTADO (COD_ESTADO)
	CONSTRAINT FK_F_EXPLORACAO_RECLASSIFICACAO
		FOREIGN KEY (COD_RECLASSIFICACAO) REFERENCES D_RECLASSIFICACAO (COD_RECLASSIFICACAO)
	CONSTRAINT FK_F_EXPLORACAO_SITUACAO
		FOREIGN KEY (COD_SITUACAO) REFERENCES D_SITUACAO (COD_SITUACAO)
	CONSTRAINT FK_F_EXPLORACAO_SONDA
		FOREIGN KEY (COD_SONDA) REFERENCES D_SONDA (COD_SONDA)
	CONSTRAINT FK_F_EXPLORACAO_TITULARIDADE
		FOREIGN KEY (COD_TITULARIDADE) REFERENCES D_TITULARIDADE (COD_TITULARIDADE)
	CONSTRAINT FK_F_EXPLORACAO_TEMPO
		FOREIGN KEY (DT_INICIO) REFERENCES D_TEMPO (DATA)
) 
""")

In [6]:
# Insert de dados f_exploracao de dataframe para database

f_exploracao_df.to_sql('F_EXPLORACAO', consqlite, if_exists="append", index=False, method=None)

In [7]:
# Informações de database f_exploracao

f_exploracao_db = pd.read_sql_query('SELECT * FROM F_EXPLORACAO', consqlite)
f_exploracao_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30172 entries, 0 to 30171
Data columns (total 45 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   COD_POCO                    30172 non-null  object 
 1   COD_CADASTRO_POCO           30172 non-null  object 
 2   COD_ESTADO                  30172 non-null  object 
 3   COD_BACIA                   30172 non-null  object 
 4   COD_BLOCO                   26759 non-null  object 
 5   COD_CAMPO                   26151 non-null  object 
 6   COD_SONDA                   30089 non-null  object 
 7   TERRA_MAR                   30172 non-null  object 
 8   POCO_POS_ANP                30172 non-null  int64  
 9   COD_CATEGORIA               30172 non-null  int64  
 10  COD_RECLASSIFICACAO         29012 non-null  object 
 11  COD_SITUACAO                30069 non-null  object 
 12  DT_INICIO                   30172 non-null  object 
 13  DT_TERMINO                  283